In [ ]:
# imports
import os
import yaml
import numpy as np
import pandas as pd
import theano
import lasagne
import loading
from training import *
from network import *
from experiments import run_tuned_fit, run_bulk_fit, run_tuned_experiment, run_bulk_experiment # replace later with * after cleaning
import architectures as arches

# aliases
L = lasagne.layers
nl = lasagne.nonlinearities
T = theano.tensor

# directories
headdir = os.path.expanduser('~/Google Drive/Bas Zahy Gianni - Games')
paramsdir_ = os.path.join(headdir, 'Analysis/0_hvh/Params/nnets/')
datadir = os.path.join(headdir, 'Data/model input')
resultsdir = os.path.join(headdir, 'Analysis/0_hvh/Loglik/nnets')

# loading data
data = loading.default_loader(os.path.join(datadir, '1-4 (no computer).csv'))
hvhdata = loading.default_loader(os.path.join(datadir, '0 (with groups).csv'))
Xs = np.concatenate(hvhdata[2])
ys = np.concatenate(hvhdata[3])
Ss = np.concatenate(hvhdata[4])

# load network specs
with open('arch_specs.yaml') as archfile:
    arch_dict = yaml.load(archfile)

# Prototyping

In [ ]:
names = ['g1', 'g2', 'g4', 'h1', 'h2', 'h4']
run_tuned_experiment(names)

# Bulk data train

In [ ]:
names = ['g1', 'g2', 'g4', 'h1', 'h2', 'h4']
run_bulk_experiment(names[:2])

# Fake news!

Train on fake data from another model.

Fit best networks on fake data

In [ ]:
fake_data = loading.default_loader(os.path.join(datadir, 'fake news (with groups).csv'))
trainer = DefaultTrainer(stopthresh=50, print_interval=25)
name = 'h4'
architecture = arch_dict[name]
print(arch_dict[name])
archfunc = getattr(arches, arch_dict[name]['type'])
arch = lambda input_var=None: archfunc(input_var, **architecture['kwargs'])
net_list = trainer.train_all(architecture=arch, data=fake_data, seed=985227)

In [ ]:
idx_test_map = {0: 5, 1: 1, 2: 2, 3: 3, 4: 4}
hd = hvhdata[0]
hXs = hvhdata[2]
hys = hvhdata[3]
hidxs = hvhdata[1]
hP = pd.DataFrame(index=hd.index, columns=list(range(36)))

for i, net in enumerate(net_list):
    test_group = idx_test_map[i]
    test_loc = hd['group'] == test_group
    
    nll = net.itemized_test_fn(hXs[test_group-1], hys[test_group-1])
    pred = net.output_fn(hXs[test_group-1])
    hd.loc[test_loc, 'nll'] = nll
    hP.loc[test_loc] = pred
    
    fname = '{} {} split fake data.npz'.format('bulk_h4', i)
    net.save_params(os.path.join(paramsdir_, 'bulk_h4', fname))

hd.to_csv(os.path.join(resultsdir, 'fake_nlls.csv'), index=False)
hdmeans = hd.pivot_table(index='subject', values='nll')

hdmeans.mean()

In [ ]:
fd = fake_data[0]
fXs = fake_data[2]
fys = fake_data[3]
fidxs = fake_data[1]
fP = pd.DataFrame(index=fd.index, columns=list(range(36)))

for i, net in enumerate(net_list):
    test_group = idx_test_map[i]
    test_loc = fd['group'] == test_group
    
    nll = net.itemized_test_fn(fXs[test_group-1], fys[test_group-1])
    pred = net.output_fn(fXs[test_group-1])
    fd.loc[test_loc, 'nll'] = nll
    fP.loc[test_loc] = pred
    
    fname = '{} {} split fake data.npz'.format('fake_h4', i)
    net.save_params(os.path.join(paramsdir_, 'fake_h4', fname))

fd.to_csv(os.path.join(resultsdir, 'fake_nlls.csv'), index=False)
fdmeans = fd.pivot_table(index='subject', values='nll')

fdmeans.mean()

In [ ]:
fP.to_csv(os.path.join(resultsdir, 'fake_predictions.csv'), index=False)

## Subject tuning

Usually doesn't work (not enough data even for very simple classifier layers)

In [ ]:
dafiname = os.path.join(datadir, '0 (with groups).csv')
subject_data = [loading.default_loader(dafiname, subject=s) for s in range(40)]
arch = archs[archname]

In [ ]:
print([len(s[0]) for s in subject_data])

In [ ]:
for i in range(5):
    pafiname = '{} {} split agg fit exp 1-4.npz'.format(archname, i)
    prenet = Network(arch)
    prenet.load_params(os.path.join(paramsdir, pafiname))
    params = L.get_all_param_values(prenet.net)
    print('PREFIT {}\n'.format(i))
    
    for s in range(40):
        sdata = subject_data[s]
        num_obs = len(sdata[0])
        bs = num_obs//5
#         if num_obs > 50:
        tuner = FineTuner(stopthresh=10, batchsize=bs)
        print('SUBJECT {}\n'.format(s))
        
        for j in range(5):
            fname = '{} {} agg fit exp 1-4 {} subject {} tune fit exp 0'.format(archname, i, s, j)
            net = tuner.train_all(architecture=arch, data=sdata, split=j, startparams=params, freeze=True)
            net.save_params(os.path.join(paramsdir, fname))

## Data aggregation

doesn't need run more than once

In [ ]:
datafilenames = ['0 (with groups)', '1 (with computer)', '2 (with computer)', '3 (with computer)', '4']
datafilenames = [os.path.join(datadir, fname + '.csv') for fname in datafilenames]
colnames = ['subject', 'color', 'bp', 'wp', 'zet', 'rt']

e0 = pd.read_csv(datafilenames[0], names=colnames+['splitno'])
e1 = pd.read_csv(datafilenames[1], names=colnames)
e2 = pd.read_csv(datafilenames[2], names=colnames)
e3 = pd.read_csv(datafilenames[3], names=colnames+['task', 'taskorder', 'session'])
e4 = pd.read_csv(datafilenames[4], names=colnames+['timecondition'])
Es = [e1, e2, e3, e4]
for i, e in enumerate(Es[1:]):
    e['subject'] = e['subject'] + Es[i-1].loc[Es[i-1]['subject']<1000, 'subject'].max()

A = pd.concat([e[colnames] for e in [e1, e2, e3, e4]])

groups = np.arange(len(A))%5 + 1
np.random.seed(100001)
np.random.shuffle(groups)
A['group'] = groups

A.to_csv(os.path.join(datadir, '1-4.csv'), encoding='ASCII', header=False, index=False)
A.loc[A['subject']<1000, :].to_csv(
    os.path.join(datadir, '1-4 (no computer).csv'), 
    encoding='ASCII', header=False, index=False
)

In [ ]:
# this is for another training scheme, using preassigned groups in both hvh and other data
bulkdata_df = pd.concat([data[0], hvhdata[0]])
bulkdata_df.to_csv(os.path.join(datadir, 'bulk.csv'), index=False, header=False)
bulkdata = loading.default_loader(os.path.join(datadir, 'bulk.csv'))

Add groups to fake data

In [ ]:
fd_ = pd.read_csv(os.path.join(datadir, 'fake news.csv'), names=['subject', 'color', 'bp', 'wp', 'zet', 'rt'])

groups = np.arange(len(fd_)) % 5 + 1
np.random.shuffle(groups)
fd_['group'] = groups
fd_.to_csv(os.path.join(datadir, 'fake news (with groups).csv'), encoding='ASCII', header=False, index=False)